In [11]:
from bs4 import BeautifulSoup
import requests
import json
import urllib
import time 
import pandas as pd
from glob import glob
from collections import ChainMap


def get_selected_cols_df(df, cols):
    '''
    cols : list of str 
    '''
    return df[cols]

url = 'https://www.fedex.com/trackingCal/track'
class FedExShipmentTracking:
    def __init__(self, df, col_name = 'Pro No', verbose = True):
        self.df = df
        self.shipment_ids = self.df[col_name].values
        self.verbose = verbose
        self.url = 'https://www.fedex.com/trackingCal/track'
        self.headers = {
                    'Connection': 'keep-alive',
                    'Pragma': 'no-cache',
                    'Cache-Control': 'no-cache',
                    'Accept': '*/*',
                    'Accept-Encoding': 'gzip, deflate, br',
                    'Accept-Language': 'en-US,en;q=0.9',
                    "Accept" : "application/json, text/javascript, */*; q=0.01",
                    'Connection': 'keep-alive',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
                    'Host': 'www.fedex.com',
                    'Origin': 'https://www.fedex.com',

                    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
                    'X-Requested-With' : "XMLHttpRequest",
                    'Cookie': 'fdx_cbid=30036807161594754925222750021081; fdx_locale=en_US; fdx_redirect=en-us; cc_path=us; s_ecid=MCMID%7C91240026263101807612795946297234918876; _abck=E6BF6C129F738D2B9D1C7477355EE277~0~YAAQL5EvFwBtVC1zAQAAAwvNTgSEKQJaNIT46+OyW/Rl1EiG2zsHB8tUiwjmoECJgd2c5DwAVVZJqwRE1WyaZUW6CBEIrzjTPlAfDtIA0eWI1PZx8cmmPMA8K043L/XtLVvra0HYHOpMsVBS/T9heRtUPhD61LP9fCtzlX8ASW2c/LueOQF0WnVI6fhEBpzT7lxVey3V7rIXOIdQx785tSS1+3g9TOPU74tgm0TvJyIpOTjLhX27RoU7wo+mzSD5HKwhzKC3pckP0hjKF1EGAXRyh7NQTD4m08aQDFxo0f0BS7BbSbLuVXHDuKdgHLRb5nJkiLs4~-1~-1~-1; _gcl_au=1.1.494437621.1594754928; tracking_locale=en_US; wdpl_id=30036807161594754925222750021081_1594754941226; optimizelyEndUserId=oeu1594842710172r0.13632731098668605; s_vi=[CS]v1|2F8820BB8515E1A5-6000073AEC279C6F[CE]; siteDC=edc; mbox=PC#1595273312475-908373.35_0#1596830577|session#1595620934343-546912#1595622837; Rbt=f0; AMCVS_1E22171B520E93BF0A490D44%40AdobeOrg=1; Nina-nina-fedex-session=%7B%22loginStatus%22%3A%22loggedOut%22%2C%22locale%22%3A%22en_us%22%2C%22lcstat%22%3Afalse%7D; s_sq=%5B%5BB%5D%5D; xacc=US; bm_sz=4537EBE6509A5D440AD47ED3F80C79BE~YAAQr3lGaOb59X9zAQAAPkn1iwjilFEKU/Hxbvqfeprv1pCh3PT2Me3bTAJQ12QgCNOtfHlxS+XK5Fa+X0LlfEm6kIgpH+Dlm9NQ0ypUeJ9BwqNwBuXI2IP6RGDvdEnsLnYg9JbJni8A/iP8YVPtGzSLZtucUOOqxj45znRIaMvvfQ7HLxOikNDq/2LQ4us=; ak_bmsc=CD81CB8828BE3D10205B58F77A78AF9F684679AF5C7D00006EAF1D5F61860971~plb5JNNd4MRJ+ZVIxorPq7NA1gFLL5TFJ/LEr5c2hx86FWjsmi7lF08BdqCL5Ee7PLBqfubVT/xzkPj+sl4riDqFyHe68mlzdT8V7t3XgyUTkNeg/eFURmr63IljjawrpW2KEswRN4XDAt1fnGzl3jEGV6PFsD+M6ECn5Cswi/rXsj0+axDVvBsXBdmloZeogYOnwKaYvTpNtxtPyYwrEzrRckCOGkkjzC/xtoyE1o6lID6AbzuMtxQvUEGAiqoQg5; AMCV_1E22171B520E93BF0A490D44%40AdobeOrg=1075005958%7CMCIDTS%7C18470%7CMCMID%7C91240026263101807612795946297234918876%7CMCAAMLH-1596385774%7C7%7CMCAAMB-1596385774%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1595788174s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.4.1; isTablet=false; isMobile=false; isWireless=false; level=test; aemserver=Prod-c0015884.prod.cloud.fedex.com; s_pers=%20s_vnum%3D1595822400191%2526vn%253D1%7C1595822400191%3B%20gpv_pageName%3Dus%252Fen%252Ffedex%252Funified%252Fsummarypage%7C1595782777780%3B%20s_nr%3D1595780977791-Repeat%7C1627316977791%3B%20s_invisit%3Dtrue%7C1595782777802%3B%20s_dfa%3Dfedexglbl%252Cfedexus%7C1595782797031%3B; s_sess=%20s_visit%3D1%3B%20SC_LINKS%3D%3B%20s_cc%3Dtrue%3B%20s_ppv%3Dus%252Fen%252Ffedex%252Funified%252Fsummarypage%252C89%252C89%252C1461%3B%20setLink%3D%3B; bm_sv=4EFB739A657166D148018747EECAE444~EEeDC8RhatPCxiJEcuvlyKw8uW9YjuNB0LsSrvXfL/4RUZTVIwcQhJSOOddouq0QvxUkB0laCZWErSpRDgmVOcGGbYaEHeDEh/JdTumXJZ3ol/DQnfjjhX6/zG1XHpRXTeqIIk+l98DMBDWQKTaMT4fzKmyVbors5C8i0jX6GsA='
                    } 

    def track(self, max_amount_track = 30):
        print(" start !!!!")
        print("we have {} fedex tracking number".format(len(self.df)))
        final_data = []
        for idx in range(0, len(self.df),max_amount_track):
            if idx + 30 <= len(self.df):
                data = self.encode(self.shipment_ids[idx:idx+30])
                response = self.crawl(data)
            else:
                data = self.encode(self.shipment_ids[idx:])
                response = self.crawl(data)
            
            bt_result = self.parse(response)
            final_data.extend(bt_result)
            time.sleep(10)
        return final_data
  
    def encode(self,ids):
        form_data = {"data":{"TrackPackagesRequest":{
                    "appType":"WTRK",
                    "appDeviceType":"DESKTOP",
                    "supportHTML":True,
                    "supportCurrentLocation":True,
                    "uniqueKey":"",
                    "processingParameters":{},
                    "trackingInfoList":[{"trackNumberInfo":{"trackingNumber":str(i),"trackingQualifier":"","trackingCarrier":""}} for i in ids]}
                            },
                    "action":"trackpackages",
                    "locale":"en_US",
                    "version":"1",
                    "format":"json",
                    }

        data = 'data='+ json.dumps(form_data['data'],separators=(',',':')) + '&action=trackpackages&locale=en_US&version=1&format=json'
        return data
        
    def crawl(self,data):
        res_id= requests.post(url,data=data,headers = self.headers)
        respond = res_id.json()
        return respond
    
    def parse(self, respond): ## list of dict 
        packages  = respond['TrackPackagesResponse'].get('packageList',[])
        if self.verbose:
            print(f"{len(packages)} loads tracked !!!")
        batch_result = []
        for p in packages:
            data = [p['displayTrackingNbr'], p['keyStatus'], p['displayEstDeliveryDateTime'] if p['keyStatus'] != 'Delivered' else p['displayActDeliveryDateTime']]
            if self.verbose:
                print(data)
                
            batch_result.append(
                {data[0]:{
                    "Action":data[1],
                    "Status" : data[2],
                }}
            )
            
        return batch_result


FEDEX_CARRIER_CODE = ['FDEN','FDEN_EXP','FXFE']
FILE_PATH = glob(".\data\IOD_MONITORING_DETAIL_US_*.csv")[-1]
df_origin = pd.read_csv(FILE_PATH)
df = df_origin[df_origin['Carrier Code'].isin(FEDEX_CARRIER_CODE)]

test = FedExShipmentTracking(df)    
result = test.track()
result = dict(ChainMap(*result))

 start !!!!
we have 1580 fedex tracking number
30 loads tracked !!!
['187887450070', 'Label created', 'Pending']
['906213154170', 'Delivery exception', 'Pending']
['906213194612', 'In transit', '7/27/2020 5:00 pm']
['906213187893', 'In transit', '7/27/2020 4:30 pm']
['906213187919', 'In transit', '7/27/2020 4:30 pm']
['906213187805', 'In transit', '7/27/2020 4:30 pm']
['906213187985', 'In transit', '7/27/2020 4:30 pm']
['906213187849', 'In transit', '7/27/2020 4:30 pm']
['906213187871', 'In transit', '7/27/2020 4:30 pm']
['906213187920', 'In transit', '7/27/2020 4:30 pm']
['906213188000', 'In transit', '7/27/2020 4:30 pm']
['906213187996', 'In transit', '7/27/2020 4:30 pm']
['906213187941', 'In transit', '7/27/2020 4:30 pm']
['906213187963', 'In transit', '7/27/2020 4:30 pm']
['906213187974', 'In transit', '7/27/2020 4:30 pm']
['187887472991', 'In transit', '7/27/2020']
['187887473082', 'In transit', '7/28/2020 4:30 pm']
['906213188087', 'Label created', '7/27/2020 3:00 pm']
['90621319

30 loads tracked !!!
['5402065222', 'In transit', '7/28/2020']
['5402065163', 'In transit', '7/29/2020']
['5402065233', 'In transit', '7/28/2020']
['5402065211', 'In transit', '7/27/2020']
['5402065200', 'In transit', '7/28/2020']
['5402065502', 'In transit', '7/27/2020']
['5402065572', 'In transit', 'Pending']
['5402065745', 'In transit', '7/27/2020']
['5402066364', 'In transit', 'Pending']
['5402066036', 'In transit', '7/29/2020']
['5402065911', 'In transit', 'Pending']
['5402065922', 'In transit', 'Pending']
['5402065966', 'Delivery exception', 'Pending']
['5402067344', 'In transit', '7/28/2020']
['5402066552', 'Picked up', '7/28/2020']
['5402067005', 'In transit', '7/29/2020']
['5402066272', 'In transit', '7/31/2020']
['5402066165', 'In transit', '7/31/2020']
['5402066106', 'In transit', '7/28/2020']
['5402066132', 'In transit', '7/29/2020']
['5402066154', 'In transit', '7/31/2020']
['5402066261', 'In transit', '7/31/2020']
['5402066121', 'In transit', '7/31/2020']
['5402066110', '

30 loads tracked !!!
['5402070133', 'In transit', 'Pending']
['5402070225', 'In transit', '7/28/2020']
['5402070310', 'In transit', '7/30/2020']
['5402070214', 'In transit', '7/29/2020']
['5402070240', 'In transit', '7/28/2020']
['5402070236', 'Picked up', '7/28/2020']
['5402070251', 'In transit', '7/28/2020']
['5402070321', 'In transit', '7/29/2020']
['5402070332', 'In transit', '7/28/2020']
['5402070564', 'In transit', 'Pending']
['5402070704', 'In transit', '7/29/2020']
['5402070730', 'In transit', '7/29/2020']
['5402070645', 'In transit', '7/29/2020']
['5402070656', 'Picked up', '7/30/2020']
['5402070774', 'In transit', '7/27/2020']
['5402071452', 'Initiated', 'Pending']
['5402071441', 'Initiated', 'Pending']
['5402071430', 'Initiated', 'Pending']
['172328507560', 'In transit', '7/27/2020']
['172328508177', 'In transit', '7/27/2020']
['172328508122', 'In transit', '7/27/2020']
['172328509060', 'In transit', '7/27/2020']
['172328508074', 'In transit', '7/27/2020']
['172328508464', '

30 loads tracked !!!
['908952843230', 'In transit', '7/27/2020']
['908952843296', 'In transit', '7/28/2020 4:30 pm']
['908952843285', 'In transit', '7/28/2020 4:30 pm']
['192470046253', 'In transit', 'Pending']
['192470050197', 'In transit', 'Pending']
['130473744158', 'In transit', 'Pending']
['187887447144', 'Delivery exception', 'Pending']
['130473746389', 'Delivery exception', '7/29/2020']
['192470094678', 'Delivery exception', 'Pending']
['192470096041', 'Delivery exception', 'Pending']
['192470096133', 'In transit', 'Pending']
['187887450997', 'In transit', 'Pending']
['192470104926', 'In transit', 'Pending']
['192470100405', 'Delivery exception', 'Pending']
['185280306300', 'In transit', 'Pending']
['192470105164', 'In transit', 'Pending']
['187887452670', 'In transit', '7/27/2020']
['130473748234', 'Delivery exception', 'Pending']
['130473748245', 'Ready for pickup', 'Pending pickup']
['130473748153', 'In transit', 'Pending']
['192470106870', 'In transit', 'Pending']
['10849272

30 loads tracked !!!
['906213178502', 'In transit', '7/28/2020']
['906213178156', 'In transit', '7/28/2020']
['906213177881', 'In transit', '7/28/2020']
['906213178925', 'In transit', '7/27/2020']
['185280314694', 'In transit', '7/27/2020']
['906213179391', 'In transit', '7/28/2020']
['906213179119', 'In transit', '7/27/2020']
['906213179417', 'In transit', '7/28/2020']
['906213179233', 'In transit', '7/27/2020']
['906213179564', 'In transit', '7/27/2020']
['906213179369', 'In transit', '7/27/2020']
['906213179575', 'In transit', '7/27/2020']
['906213179060', 'In transit', '7/27/2020']
['906213179163', 'In transit', '7/27/2020']
['906213179130', 'In transit', '7/27/2020']
['906213179108', 'In transit', '7/27/2020']
['906213179586', 'In transit', '7/27/2020']
['906213179299', 'In transit', '7/28/2020']
['906213179406', 'In transit', '7/27/2020']
['906213179597', 'In transit', '7/28/2020']
['906213179440', 'In transit', '7/27/2020']
['906213179439', 'In transit', '7/28/2020']
['906213179

30 loads tracked !!!
['906213183913', 'In transit', 'Pending']
['906213180064', 'In transit', '7/28/2020']
['906213180075', 'In transit', '7/28/2020']
['906213180145', 'In transit', '7/28/2020']
['906213180086', 'In transit', '7/28/2020']
['906213181998', 'In transit', '7/28/2020']
['906213182479', 'In transit', '7/27/2020']
['906213182056', 'In transit', '7/28/2020']
['906213181255', 'In transit', '7/28/2020']
['906213181987', 'In transit', '7/28/2020']
['906213195295', 'In transit', '7/31/2020']
['906213182619', 'In transit', '7/28/2020']
['906213182711', 'In transit', '7/29/2020']
['906213194299', 'In transit', '7/28/2020']
['906213185125', 'In transit', '7/27/2020']
['906213182620', 'In transit', '7/28/2020']
['906213182527', 'In transit', '7/28/2020']
['906213182847', 'In transit', '7/28/2020']
['906213182696', 'In transit', '7/27/2020']
['906213182641', 'In transit', '7/28/2020']
['906213187014', 'In transit', '7/27/2020']
['906213187698', 'In transit', '7/29/2020']
['90621318760

30 loads tracked !!!
['906213183902', 'In transit', '7/28/2020']
['906213184714', 'In transit', '7/28/2020']
['906213184920', 'In transit', '7/29/2020']
['906213184747', 'In transit', '7/28/2020']
['906213183730', 'In transit', '7/28/2020']
['906213183133', 'In transit', '7/28/2020']
['906213183442', 'In transit', '7/28/2020']
['906213183339', 'In transit', '7/28/2020']
['906213183041', 'In transit', '7/28/2020']
['906213185088', 'In transit', '7/28/2020']
['906213195089', 'In transit', '7/27/2020']
['906213194417', 'In transit', '7/28/2020']
['906213183946', 'In transit', '7/28/2020']
['130473755177', 'In transit', '7/27/2020']
['130473755410', 'In transit', '7/27/2020']
['906213183660', 'In transit', 'Pending']
['906213194211', 'In transit', '7/27/2020']
['906213183166', 'In transit', '7/28/2020']
['906213184004', 'In transit', '7/27/2020']
['906213185011', 'In transit', '7/28/2020']
['906213183810', 'In transit', '7/28/2020']
['906213184894', 'In transit', '7/28/2020']
['90621318352

30 loads tracked !!!
['906213192219', 'In transit', '7/28/2020']
['906213192138', 'In transit', '7/28/2020']
['906213192252', 'In transit', '7/28/2020']
['906213192403', 'In transit', '7/29/2020']
['906213195262', 'In transit', '7/27/2020']
['906213188032', 'In transit', '7/29/2020']
['906213192171', 'In transit', '7/28/2020']
['906213195273', 'In transit', '7/30/2020']
['906213188010', 'In transit', '7/28/2020']
['906213188043', 'In transit', '7/29/2020']
['185280318440', 'In transit', '7/28/2020']
['185280318277', 'In transit', '7/27/2020']
['185280318266', 'In transit', '7/28/2020']
['185280318255', 'In transit', '7/28/2020']
['185280318461', 'In transit', '7/28/2020']
['185280318623', 'In transit', '7/28/2020']
['185280318634', 'In transit', '7/28/2020']
['185280318612', 'In transit', '7/29/2020']
['185280318601', 'In transit', '7/27/2020']
['185280318531', 'In transit', '7/28/2020']
['185280318244', 'In transit', '7/28/2020']
['185280318553', 'In transit', '7/28/2020']
['185280318

In [40]:
df['Action'] = df['Pro No'].apply(lambda x : result[x]['Action'])
df['Status'] = df['Pro No'].apply(lambda x : result[x]['Status'])
df.to_csv("report.csv",index=False)

C:\Users\marco.wang\Miniconda3\envs\pantos\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\marco.wang\Miniconda3\envs\pantos\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
